In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import numpy as np
import tempfile
import tensorflow as tf

from tf_rl.controller import DiscreteDeepQ, HumanController
from tf_rl.simulation import KarpathyGame
from simulation.grid_game import GridGame
from tf_rl import simulate
from tf_rl.models import MLP

# from __future__ import print_function

hero is at pos Point2(1.00, 0.00)
target is at pos Point2(0.00, 2.00)


In [3]:
LOG_DIR = tempfile.mkdtemp()
print(LOG_DIR)

/tmp/tmpZmvTLZ


In [4]:
def simulate(simulation,
             controller= None, training=True):

    # state transition bookkeeping
    ii = -1
    counter_win = 0
    counter_edge = 0
    counter_lost = 0
    print 'let the game begin'
    while ii < 100000000000:
        log_interval = 1000
        if ii% log_interval == 0:
            print(counter_win-(counter_lost + counter_edge))/float(10000)
            counter_win = 0
            counter_edge = 0
            counter_lost = 0
        simulation.reinitialize()
        last_observation = None
        last_action      = None
        game_ends = False
        ii += 1
        # Play one round of game
        while game_ends == False:
            
            new_observation = simulation.observe()
            reward          = simulation.collect_reward()
            if reward == 1 or reward == -1:
                game_ends = True
                if reward == 1:
                    counter_win +=1
                else:
                    counter_edge += 1
            
            if simulation.action_counter == 20:
                game_ends = True
                counter_lost += 1
                reward = -1
    
                
            # store last transition
            if last_observation is not None:
                controller.store(last_observation, last_action, reward, new_observation)

            # act
            new_action = controller.action(new_observation)
            simulation.perform_action(new_action)



            #train
            if training:
                controller.training_step()

            # update current state as last state.
            last_action = new_action
            last_observation = new_observation
            






In [5]:
g = GridGame(board=(4, 4))
print(g)

hero is at pos Point2(2.00, 2.00)
target is at pos Point2(2.00, 3.00)


In [6]:
 
print('test')
# Tensorflow business - it is always good to reset a graph before creating a new controller.
tf.ops.reset_default_graph()
session = tf.InteractiveSession()

# This little guy will let us run tensorboard
#      tensorboard --logdir [LOG_DIR]
journalist = tf.train.SummaryWriter(LOG_DIR)

# Brain maps from observation to Q values for different actions.
# Here it is a done using a multi layer perceptron with 2 hidden
# layers
brain = MLP([g.observation_size,], [200, 200, g.num_actions], 
            [tf.tanh, tf.tanh, tf.identity])

# The optimizer to use. Here we use RMSProp as recommended
# by the publication
optimizer = tf.train.RMSPropOptimizer(learning_rate= 0.00001, decay=0.9)

# DiscreteDeepQ object
current_controller = DiscreteDeepQ(g.observation_size, g.num_actions, brain, optimizer, session,
                                   discount_rate=0.99, exploration_period=5000, max_experience=10000, 
                                   store_every_nth=1, train_every_nth=4,
                                   summary_writer=journalist, random_action_probability=0.0)

session.run(tf.initialize_all_variables())
session.run(current_controller.target_network_update)
# graph was not available when journalist was created 

journalist.add_graph(session.graph_def)

test


In [7]:
print('test')
simulate(g, controller=current_controller)

test
let the game begin
0.0001
exploration period over
-0.0459
-0.0355
-0.009
0.0079
0.0066
0.0078
0.0136
0.0036
0.019
0.0284
0.0282
0.0266
0.034
0.0336
0.031
0.0338
0.0356
0.0372
0.0346
0.035
0.038
0.0342
0.0376
0.0404
0.0368
0.0418
0.0402
0.0352
0.038
0.036
0.0384
0.043
0.0372
0.0352
0.0388
0.0409
0.0338
0.0368
0.0374
0.0408
0.0361
0.0412
0.033
0.0332
0.0318
0.0368
0.0412
0.0398
0.0432
0.0372
0.042
0.0404
0.039
0.0396
0.0368
0.0358
0.0366
0.0388
0.0334
0.0374
0.0382
0.039
0.0384
0.0378
0.0382
0.0366
0.0448
0.0408
0.0408
0.0356
0.0368
0.0394
0.0304
0.0446
0.04
0.0372
0.0364
0.0406
0.0386
0.0396
0.0338
0.0372
0.0374
0.0394
0.0382
0.04
0.037
0.0394
0.0412
0.0372
0.0412
0.0382
0.0414
0.0394
0.0418
0.0364
0.036
0.035
0.0334
0.0378
0.0372
0.04
0.0368
0.0362
0.041
0.036
0.0374
0.0394
0.035
0.0384
0.0372
0.0394
0.0378
0.0378
0.0388
0.036
0.0396
0.0234
0.027
0.0256
0.0302
0.0248
0.0301
0.0298
0.0278
0.03
0.034
0.0272
0.0256
0.0312
0.0266
0.0316
0.0262
0.0222
0.022
0.027
0.0268
0.0298
0.029
0.

KeyboardInterrupt: 

In [ ]:
0 < g.position.y < g.board.y

In [ ]:
g.position.x